[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-2/chatbot-external-memory.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239440-lesson-6-chatbot-w-summarizing-messages-and-external-memory)

# 具备消息总结与外部数据库记忆的聊天机器人

## 回顾

我们已经讲过如何自定义图的状态（state）模式以及 reducer（归约器）。

也展示了多种在图状态中裁剪或过滤消息的技巧。

我们用这些概念实现过一个带记忆的聊天机器人，它会生成对话的滚动摘要。

## 目标

但是，如果我们希望聊天机器人拥有“可以无限期持久化”的记忆怎么办？

现在，我们将引入一些更高级的检查点（checkpointer），它们支持外部数据库。

这里我们展示如何使用 [Sqlite 作为 checkpointer](https://docs.langchain.com/oss/python/langgraph/persistence#checkpointer-libraries)，当然还可以选择 Postgres 等其他实现！

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph-checkpoint-sqlite langchain_core langgraph langchain_openai

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from rich import print as rprint

# 获取项目根目录（当前文件所在目录的上一级）
project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
env_path = project_root / '.env'

# 加载 .env 文件
load_dotenv(dotenv_path=env_path)

required_vars = ['OPENAI_API_KEY', 'OPENAI_API_BASE', 'MODEL_NAME', "LANGSMITH_API_KEY", "LANGSMITH_TRACING", "LANGSMITH_PROJECT"]
missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    rprint(f"[red]警告: 以下环境变量未在 .env 文件中配置: {', '.join(missing_vars)}[/red]")
else:
    rprint(f"[green]✓ 环境变量已从 {env_path} 成功加载[/green]")

## Sqlite

一个好的起步方式是使用 [SqliteSaver checkpointer](https://docs.langchain.com/oss/python/langgraph/persistence#checkpointer-libraries)。

Sqlite 是一个[小巧、快速、非常流行](https://x.com/karpathy/status/1819490455664685297)的 SQL 数据库。

如果我们提供 `":memory:"`，它会创建一个内存中的 Sqlite 数据库。

In [ ]:
import sqlite3
# In memory
conn = sqlite3.connect(":memory:", check_same_thread = False)

但是，如果我们提供一个数据库路径，它就会为我们创建一个真正的数据库文件！

In [ ]:
# pull file if it doesn't exist and connect to local db
!mkdir -p state_db && [ ! -f state_db/example.db ] && wget -P state_db https://github.com/langchain-ai/langchain-academy/raw/main/module-2/state_db/example.db

db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

In [ ]:
# Here is our checkpointer 
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

我们来重新定义一下聊天机器人。

In [ ]:
from typing_extensions import Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState


model = ChatOpenAI(model="gpt-4o",temperature=0)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
    
    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State)-> Literal ["summarize_conversation",END]:
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

现在，我们只需要用刚才的 sqlite checkpointer 重新编译即可。

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
graph = workflow.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

现在，我们可以多次调用这个图。

In [ ]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Start conversation
input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

我们来确认一下状态确实被保存在本地。

In [ ]:
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

### 状态持久化

使用 Sqlite 这样的数据库意味着状态是持久化的！

例如：我们可以重启 notebook 的内核，然后仍然可以从磁盘上的 Sqlite 数据库加载之前的状态。

In [ ]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

## Studio

**⚠️ 注意**

自录制这些视频以来，我们已更新 Studio，使其现在可以在本地运行并通过浏览器访问。这是替代视频中所示桌面应用（Desktop App）的首选运行方式。它现在被称为 _LangSmith Studio_，而不是之前的 _LangGraph Studio_。详细的设置说明在课程开头的 “Getting Setup” 指南中。你可以在[这里](https://docs.langchain.com/langsmith/studio)找到 Studio 的描述，本地部署的具体细节在[这里](https://docs.langchain.com/langsmith/quick-start-studio#local-development-server)。  
要启动本地开发服务器，请在本模块的 `/studio` 目录中运行：

```
langgraph dev
```

你应当看到如下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

在浏览器中打开上面显示的 **Studio UI** URL。
加载 Studio 中的 `chatbot`，它使用 `module-2/studio/chatbot.py`，并在 `module-2/studio/langgraph.json` 中进行了配置。